In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
path = 'D:\\資料處理\\SBAdata\\SBAnational.csv'
df = pd.read_csv(path)
df_copy = df.copy()
display(df.iloc[:5,:11])
display(df.iloc[:5,11:])

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240


,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [3]:
df_copy.shape

(899164, 27)

In [4]:
df2 = df.iloc[:50, :]

In [5]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [6]:
# 處理日期
d_cols = ['ChgOffDate', 'DisbursementDate', 'ApprovalDate']
df_copy[d_cols] = df_copy[d_cols].fillna('xx-xx-xx')

for col in d_cols:
    
    df_copy[col + '_year'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[2])
    df_copy[col + '_month'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[1])
    df_copy[col + '_day'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[0])

# df_copy.drop(d_cols, axis=1, inplace=True)
df_copy.drop('ApprovalFY', axis=1, inplace=True)

mon_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12, 'xx': 'xx'}

# 月份文字轉數字
df_copy['ChgOffDate_month'] = df_copy['ChgOffDate_month'].map(mon_dict)
df_copy['DisbursementDate_month'] = df_copy['DisbursementDate_month'].map(mon_dict)
df_copy['ApprovalDate_month'] = df_copy['ApprovalDate_month'].map(mon_dict)

# 年份二位轉四位
def year_2d_to_4d(x):
    try:
        if int(x) > 14:  # 代表19xx年
            x = '19' + x
        else:  # 20xx年
            x = '20' + x
    except ValueError:
        pass
    return x

year_cols = ['ChgOffDate_year', 'DisbursementDate_year', 'ApprovalDate_year']

df_copy[year_cols] = df_copy[year_cols].astype('str').applymap(year_2d_to_4d)

In [7]:
def money_clean(org_money):
    org_money = org_money.strip().split('.')[0]
    org_money = org_money.strip('$').replace(',', '')

    return org_money

In [8]:
money_cols = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
df_copy.loc[:, money_cols] = df_copy.loc[:, money_cols].applymap(money_clean)
df_copy[money_cols] = df_copy[money_cols].astype('int64')

In [9]:
df_copy['NewExist'] = df_copy['NewExist'].map({2: 0, 1: 1})
df_copy['UrbanRural'] = df_copy['UrbanRural'].map({2: 0, 0: -1, 1: 1})
df_copy['RevLineCr'] = df_copy['RevLineCr'].map({'Y': 1, 'N': 0})
df_copy['LowDoc'] = df_copy['LowDoc'].map({'Y': 1, 'N': 0})

In [10]:
df_copy['Industry'] = df_copy['NAICS'].astype('str').apply(lambda x: x[:2])
df_copy['Industry'] = df_copy['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin',
    '0': 'Unknown'
})

df_copy.drop('NAICS', axis=1, inplace=True)

In [11]:
df_copy = df_copy.apply(lambda x: x.replace('xx', np.nan))
df_copy = df_copy.apply(lambda x: x.replace('xx-xx-xx', np.nan))

# 缺失值處理

In [12]:
%run function.ipynb

In [13]:
# 找不到RevLineCr的缺失規律，填-1變第三個類別
df_copy['RevLineCr'].fillna(-1, inplace=True)
df_copy['RevLineCr'].value_counts()

 0.0    420288
-1.0    277479
 1.0    201397
Name: RevLineCr, dtype: int64

In [14]:
# 找不到 LowDoc的缺失規律，填-1變第三個類別
df_copy['LowDoc'].fillna(-1, inplace=True)
df_copy['LowDoc'].value_counts()

 0.0    782822
 1.0    110335
-1.0      6007
Name: LowDoc, dtype: int64

In [15]:
df_copy['MIS_Status'].value_counts()
# 19  ChgOffDate         162699 non-null  object

P I F     739609
CHGOFF    157558
Name: MIS_Status, dtype: int64

In [16]:
na_check(df_copy)  # 7/20目標

,Missing Ratio
ChgOffDate,0.819055
ChgOffDate_year,0.819055
ChgOffDate_month,0.819055
ChgOffDate_day,0.819055
DisbursementDate,0.002634
DisbursementDate_year,0.002634
DisbursementDate_month,0.002634
DisbursementDate_day,0.002634
MIS_Status,0.002221
BankState,0.001742


In [17]:
# 把target不見的抽出來
df_target_miss = df_copy[is_na_or_not(df_copy['MIS_Status'])]
df_target_miss.head(5)
display(na_check(df_target_miss))

,Missing Ratio
MIS_Status,1.000000
ChgOffDate,0.869304
ChgOffDate_year,0.869304
ChgOffDate_month,0.869304
ChgOffDate_day,0.869304
DisbursementDate,0.096645
DisbursementDate_year,0.096645
DisbursementDate_month,0.096645
DisbursementDate_day,0.096645
Bank,0.026540


In [18]:
display(df_target_miss['ChgOffPrinGr'])
df_target_miss['ChgOffPrinGr'].value_counts()

343            0
611       144461
738            0
740       142666
833            0
           ...  
892340         0
893791         0
894290         0
896318         0
898182         0
Name: ChgOffPrinGr, Length: 1997, dtype: int64

0        1740
5000        5
1000        4
2500        3
999         2
         ... 
12995       1
4807        1
714         1
716         1
49139       1
Name: ChgOffPrinGr, Length: 242, dtype: int64

In [19]:
df_pif_weird = df_copy[((df_copy['MIS_Status'] == 'CHGOFF')&(df_copy['ChgOffPrinGr'] == 0)) | 
                       ((df_copy['MIS_Status'] == 'P I F')&(df_copy['ChgOffPrinGr'] != 0))]
df_pif_weird[['Name', 'ChgOffPrinGr', 'MIS_Status', 'FranchiseCode']].sort_values(by='Name', ascending=True).head(30)
df_pif_weird['Name'].value_counts()

GTP INC                           3
SCOTT AVIATION                    3
THE SWEATER COMPANY               3
AAMCO TRANSMISSIONS               3
KOALA T KIDS PRESCHOOL            2
                                 ..
The Main China Corporation        1
E & A Trucking Co., L.L.C.        1
LEATHER TOGETHER INC              1
SHAHRAM KHLAL PH D & ASSOCIATE    1
Christi R. Alexander dba Na-Na    1
Name: Name, Length: 5489, dtype: int64

In [29]:
df_copy[df_copy['Name'] == 'AAMCO TRANSMISSIONS'][['Name', 'City', 'ChgOffPrinGr', 'MIS_Status', 'FranchiseCode', 'ApprovalDate', 'ApprovalDate_year', 'ChgOffDate', 'ChgOffDate_year']].sort_values(by=['City', 'ChgOffDate_year'], ascending=False).head(50)

,Name,City,ChgOffPrinGr,MIS_Status,FranchiseCode,ApprovalDate,ApprovalDate_year,ChgOffDate,ChgOffDate_year
222238,AAMCO TRANSMISSIONS,ZELIENOPLE,160394,CHGOFF,1350,13-Dec-06,2006,17-Jun-11,2011
241718,AAMCO TRANSMISSIONS,YUBA CITY,224374,CHGOFF,1350,14-Mar-07,2007,1-Jun-10,2010
241719,AAMCO TRANSMISSIONS,YUBA CITY,24833,CHGOFF,1350,14-Mar-07,2007,11-Jun-10,2010
830303,AAMCO TRANSMISSIONS,WYOMING,0,P I F,1,16-Jan-96,1996,NaN,NaN
816809,AAMCO TRANSMISSIONS,WILMINGTON,0,P I F,1350,8-Apr-05,2005,NaN,NaN
313389,AAMCO TRANSMISSIONS,WEST SENECA,0,P I F,1350,10-Sep-99,1999,NaN,NaN
540332,AAMCO TRANSMISSIONS,WEST PALM BEACH,167513,CHGOFF,1350,29-Mar-02,2002,5-Jul-06,2006
865490,AAMCO TRANSMISSIONS,WASHINGTON,0,P I F,1,19-Jun-96,1996,NaN,NaN
892927,AAMCO TRANSMISSIONS,WARWICK,0,P I F,1350,16-Jan-97,1997,NaN,NaN
733658,AAMCO TRANSMISSIONS,TITUSVILLE,0,P I F,1350,28-Jul-04,2004,NaN,NaN


In [21]:
df_copy = df_copy[~is_na_or_not(df_copy['MIS_Status'])]
df_copy.index = range(len(df_copy))
df_copy

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,28-Feb-97,84,4,...,NaN,NaN,NaN,1999,2.0,28,1997,2,28,Retail_trade
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,28-Feb-97,60,2,...,NaN,NaN,NaN,1997,5.0,31,1997,2,28,Accom/Food_serv
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,28-Feb-97,180,7,...,NaN,NaN,NaN,1997,12.0,31,1997,2,28,Healthcare/Social_assist
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,28-Feb-97,60,2,...,NaN,NaN,NaN,1997,6.0,30,1997,2,28,Unknown
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,28-Feb-97,240,14,...,NaN,NaN,NaN,1997,5.0,14,1997,2,28,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897162,9995573004,FABRIC FARMS,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,27-Feb-97,60,6,...,NaN,NaN,NaN,1997,9.0,30,1997,2,27,Retail_trade
897163,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,27-Feb-97,60,6,...,NaN,NaN,NaN,1997,10.0,31,1997,2,27,Retail_trade
897164,9995613003,"RADCO MANUFACTURING CO.,INC.",SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,27-Feb-97,108,26,...,NaN,NaN,NaN,1997,9.0,30,1997,2,27,Manufacturing
897165,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,27-Feb-97,60,6,...,2000,3.0,8,1997,3.0,31,1997,2,27,Unknown


In [22]:
display(df_copy.iloc[:5,:18])
display(df_copy.iloc[:5,18:])

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,28-Feb-97,84,4,0.0,0,0,1,-1,0.0,1.0,NaN
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,28-Feb-97,60,2,0.0,0,0,1,-1,0.0,1.0,NaN
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,28-Feb-97,180,7,1.0,0,0,1,-1,0.0,0.0,NaN
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,28-Feb-97,60,2,1.0,0,0,1,-1,0.0,1.0,NaN
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,28-Feb-97,240,14,1.0,7,7,1,-1,0.0,0.0,NaN


,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
0,28-Feb-99,60000,0,P I F,0,60000,48000,NaN,NaN,NaN,1999,2.0,28,1997,2,28,Retail_trade
1,31-May-97,40000,0,P I F,0,40000,32000,NaN,NaN,NaN,1997,5.0,31,1997,2,28,Accom/Food_serv
2,31-Dec-97,287000,0,P I F,0,287000,215250,NaN,NaN,NaN,1997,12.0,31,1997,2,28,Healthcare/Social_assist
3,30-Jun-97,35000,0,P I F,0,35000,28000,NaN,NaN,NaN,1997,6.0,30,1997,2,28,Unknown
4,14-May-97,229000,0,P I F,0,229000,229000,NaN,NaN,NaN,1997,5.0,14,1997,2,28,Unknown


In [23]:
# # groupby
# temp = df_copy[pd.isnull(df_copy['LowDoc'])]
# display(pd.DataFrame(temp.groupby(by=['City', 'State']).size().sort_values(ascending=False).head(30)))
# # temp2 = df_copy[df_copy['City'] == 'EL PASO']
# # display(temp2['LowDoc'].value_counts())
# temp2